In [ ]:
from s3prl.downstream.mdd.dataset import L2ArcticDataset
from s3prl.downstream.augment_utils.add_noise import AddNoise
from IPython.display import Audio, display
from pathlib import Path
import torch
import torchaudio
import numpy as np

In [ ]:
folder = Path("/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/rir/Audio")

wavs = []
srs = []

for path in folder.iterdir():
    if path.suffix != ".wav":
        continue
    wav, sr = torchaudio.load(path)
    wav = wav.squeeze()
    wav = wav / wav.norm()
    wavs.append(wav)
    srs.append(sr)
sr = srs[0]

In [ ]:
from matplotlib import pyplot as plt


def plot(wav, ax=None, **kwargs):
    if ax is not None:
        ax.plot(np.linspace(0, len(wav) / sr, wav.shape[-1]), wav, **kwargs)
    else:
        plt.plot(np.linspace(0, len(wav) / sr, wav.shape[-1]), wav, **kwargs)


plot(wavs[3][: int(0.025 * sr)])

In [ ]:
dataset = L2ArcticDataset(
    "train",
    "/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0",
    "/home/xt0r3-user/cambridge/partii/dissertation/s3prl/s3prl/s3prl/downstream/mdd/data/",
)

In [ ]:
def play_audio(waveform, sample_rate):
    waveform = waveform.numpy()
    display(Audio(waveform, rate=sample_rate))


audio = [
    torchaudio.functional.resample(dataset[0][0], orig_freq=44100, new_freq=16000),
    torchaudio.functional.resample(dataset[1][0], orig_freq=44100, new_freq=16000),
]

In [ ]:
Audio(torchaudio.functional.fftconvolve(audio[0], wavs[30]), rate=16000)

In [ ]:
Audio(torchaudio.functional.fftconvolve(audio[0], wavs[12]), rate=16000)

In [ ]:
Audio(audio[0], rate=16000)

In [ ]:
def plot_rev(x, **kwargs):
    plot(torchaudio.functional.fftconvolve(audio[0], wavs[x]), **kwargs)

In [ ]:
plt.figure(figsize=(10, 5))


def plot_shift(t):
    mult = 2000
    wav = wavs[1][239:]
    sound = audio[0][: len(audio[0]) // 4]
    if t != 0:
        sound = sound[: -t * mult]
    # if t == 0:
    #     plt.plot(audio[0][t*mult:] * wavs[3][t], alpha=(wav / wav.max()).item())
    # else:
    plt.plot(
        torch.cat([torch.zeros(t * mult), sound]),
        alpha=min(1, (wav[t].abs() / wav.max()).item() + 0.2),
    )


for i in range(0, 6):
    # [0, 1, 2, 3]
    plot_shift(i)
    plt.axis("off")
    plt.savefig(
        f"/home/xt0r3-user/cambridge/partii/dissertation/final/figs/chapter-3/reverb/rev_{i}.png",
        bbox_inches="tight",
    )
    plt.clf()

In [ ]:
wavs[1].argmax()

In [ ]:
plt.plot(audio[0][: len(audio[0]) // 4])

In [ ]:
plt.figure(figsize=(25, 5))
plt.plot(
    torchaudio.functional.fftconvolve(audio[0][: len(audio[0]) // 4], wavs[1][141:])[
        : len(audio[0]) // 4
    ]
)
plt.axis("off")
plt.savefig(
    f"/home/xt0r3-user/cambridge/partii/dissertation/final/figs/chapter-3/reverb/result.png",
    bbox_inches="tight",
)

In [ ]:
plt.figure(figsize=(25, 5))
plt.plot(audio[0][: len(audio[0]) // 4])
plt.axis("off")
plt.savefig(
    f"/home/xt0r3-user/cambridge/partii/dissertation/final/figs/chapter-3/reverb/start.png",
    bbox_inches="tight",
)

In [ ]:
plt.plot(wavs[3][141:161])

In [ ]:
plt.plot(wavs[3])

In [ ]:
wavs[3].argmax()

In [ ]:
wavs[3].max()

In [ ]:
plot_rev(3)
plt.axis("off")

In [ ]:
plt.figure(figsize=(12, 6))
ax1 = plt.subplot(2, 2, 1)
ax2 = plt.subplot(2, 2, 3)
ax3 = plt.subplot(1, 2, 2)

n = 12

plot(wavs[n][: int(0.1 * sr)], ax=ax1)
plot(audio[0], ax=ax2)
plot_rev(n, ax=ax3)

In [ ]:
rng = torch.Generator()
rng.manual_seed(1337)
n = 5
indices = torch.randint(
    0,
    len(wavs),
    (n,),
    generator=rng,
)

fig, ax = plt.subplots(figsize=(10, 2 * n))

for i, idx in enumerate(indices):
    plt.plot(torchaudio.functional.fftconvolve(audio[0], wavs[idx]))
    plt.axis("off")
    plt.savefig(f"wav_{i}.png", bbox_inches="tight")
    plt.clf()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 2))
plt.plot(audio[0], color="k")
plt.axis("off")
plt.savefig(f"wav.png", bbox_inches="tight")
plt.clf()